In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import multiprocessing
import torch.nn as nn

!pip install --no-deps git+https://github.com/lfochamon/labx.git
import labx

In [ ]:
# If you have a GPU installed and configured correctly, this code will allow the use of gpu
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

if torch.cuda.is_available():
    print(f"Using GPU: {torch.cuda.get_device_name(0)}")
    batch_size = 128
    pin_memory = True
else:
    print("No GPU found, running on CPU.")
    batch_size = 32
    pin_memory = False

In [ ]:
# Dowloading the dataset
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

CIFAR10_MEAN = (0.4914, 0.4822, 0.4465)
CIFAR10_STD = (0.2023, 0.1994, 0.2010)
CIFAR10_LABELS = [
    "airplane",
    "automobile",
    "bird",
    "cat",
    "deer",
    "dog",
    "frog",
    "horse",
    "ship",
    "truck",
]

# Define transforms for the training and testing datasets
transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize(CIFAR10_MEAN, CIFAR10_STD),
    ]
)

# Load CIFAR-10 dataset
train_dataset = datasets.CIFAR10(root="./data", train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root="./data", train=False, download=True, transform=transform)

# Create data loaders
train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=multiprocessing.cpu_count(),
    pin_memory=pin_memory,
)
test_loader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=multiprocessing.cpu_count(),
    pin_memory=pin_memory,
)


# Unnormalize data
def unnormalize(x):
    std = torch.tensor([[CIFAR10_STD]]).reshape((3, 1, 1))
    mean = torch.tensor([[CIFAR10_MEAN]]).reshape(3, 1, 1)
    return torch.clamp(x * std + mean, 0, 1)

In [ ]:
# Create model instance
model = labx.resnet18(pretrained=True, device=device)

In [ ]:
def accuracy(model):

    model.eval()
    val_loss = 0.0
    val_acc = 0
    total_val = 0

    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)

            outputs = model(inputs)
            _, preds = outputs.max(dim=1)
            total_val += targets.size(dim=0)
            val_acc += preds.eq(targets).sum().item()

    val_loss /= total_val
    val_acc = val_acc / total_val

    print(f"Test acc: {100*val_acc:.2f}%")


accuracy(model)

In [ ]:
def attack(model, input, target, epsilon, eta, iterations):
    epsilon, eta = 2.5 * epsilon, 2.5 * eta

    # Define the loss function
    criterion = nn.CrossEntropyLoss()

    # Store the history
    history = {"train_loss": [], "train_acc": []}

    model.eval()
    input.requires_grad = False
    target.requires_grad = False

    mod_input = input.clone() + epsilon * (2 * torch.rand_like(input) - 1)
    for _ in range(iterations):
        mod_input.grad = None
        mod_input = mod_input.detach()
        mod_input.requires_grad = True

        output = model(mod_input)
        loss = criterion(output, target)
        loss.backward()

        mod_input = mod_input + eta * mod_input.grad.sign()
        mod_input = input + torch.clamp(mod_input - input, -epsilon, epsilon)

        _, pred = output.max(1)

        history["train_loss"].append(loss.item())
        history["train_acc"].append(pred.eq(target))

    return mod_input, history

In [ ]:
fig, axes = plt.subplots(3, 7, figsize=(17, 6))

for n, idx in enumerate(np.random.randint(0, len(test_dataset), 7)):
    input, target = test_dataset[idx]
    input, target = input.to(device), torch.tensor([target], device=device)

    mod_input, _ = attack(
        model, input.unsqueeze(0), target, epsilon=0.03, eta=0.03 / 20, iterations=50
    )
    mod_input = mod_input.detach()

    output = model(input.unsqueeze(0))
    _, pred = output.max(1)

    output = model(mod_input)
    _, mod_pred = output.max(1)

    input_img = unnormalize(input.to("cpu")).movedim(0, 2)
    axes[0, n].imshow(input_img)
    _ = axes[0, n].set_title(CIFAR10_LABELS[pred])

    mod_input_img = unnormalize(mod_input.to("cpu").squeeze()).movedim(0, 2)
    axes[1, n].imshow(mod_input_img)
    _ = axes[1, n].set_title(CIFAR10_LABELS[mod_pred])

    difference = (input_img - mod_input_img).abs()
    axes[2, n].imshow(difference / difference.max())
    _ = axes[2, n].set_title("Difference")

for ax in axes.flatten():
    ax.axis("off")

In [ ]:
def robustness(model):

    model.eval()
    val_loss = 0.0
    val_acc = 0
    total_val = 0

    for inputs, targets in test_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        mod_inputs, _ = attack(model, inputs, targets, epsilon=0.03, eta=0.03 / 20, iterations=50)

        with torch.no_grad():
            outputs = model(mod_inputs)
            _, preds = outputs.max(dim=1)
            total_val += targets.size(dim=0)
            val_acc += preds.eq(targets).sum().item()

    val_loss /= total_val
    val_acc = val_acc / total_val

    print(f"Robust acc: {100*val_acc:.2f}%")


robustness(model)

In [ ]:
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()

epochs = 3
for epoch in range(epochs):
    train_loss = 0.0
    train_acc = 0
    total_train = 0

    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)

        mod_inputs, _ = attack(model, inputs, targets, epsilon=0.03, eta=0.01, iterations=5)
        mod_inputs = mod_inputs.detach()
        mod_inputs.requires_grad = False
        model.train()

        optimizer.zero_grad()
        outputs = model(mod_inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * inputs.size(0)
        _, preds = outputs.max(1)
        total_train += targets.size(0)
        train_acc += preds.eq(targets).sum().item()

    train_loss /= total_train
    train_acc = train_acc / total_train

    print(
        f"Epoch {epoch+1}/{epochs} - Train Loss: {train_loss:.4f}  Train Acc: {100*train_acc:.2f}%"
    )

In [ ]:
accuracy(model)
robustness(model)